# Simple RNN models

Now that I have extracted MFCC/Mel filter bank features from the audio, I would like to test the simplest neural network models. Basic RNN (GRU, LSTM) models will be tested.

Trained on GeForce GTX 1080.

So far the results on the validation set are:

| Feature | Model  | Hidden Dim  | Accuracy  |
|:--------:|:-----:|:-----------:|:---------:|
| MFCC (39d)| LSTM | 128        |  .9206 |
| MFCC (39d)| LSTM | 256        |  .9297 |
| MFCC (39d)| LSTM | 512        |  **.9356** |
| MFCC (39d)| GRU  | 128        |  .9238 |
| MFCC (39d)| GRU  | 256        |  .9316 |
| MFCC (39d)| GRU  | 512        |  .9278 |


In [2]:
import pandas as pd
import numpy as np
from collections import Counter

from keras.layers import Dense, LSTM, GRU, Activation, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from sklearn.metrics import classification_report

## load data

In [3]:
%ls preprocessed

test_mel.npy   test_spec.npy  train_mfcc.npy  valid_mel.npy   valid_spec.npy
test_mfcc.npy  train_mel.npy  train_spec.npy  valid_mfcc.npy


In [4]:
features = ["mel","mfcc"]

train, valid = {},{}

for f in features:
    train[f] = np.load("preprocessed/train_%s.npy" % f).T
    train[f] = np.swapaxes(train[f], 1,2)
    print("loaded %s" % str(train[f].shape))
for f in features:
    valid[f] = np.load("preprocessed/valid_%s.npy" % f).T
    valid[f] = np.swapaxes(valid[f], 1,2)
    print("loaded %s" % str(valid[f].shape))

loaded (51088, 99, 40)
loaded (51088, 99, 39)
loaded (6798, 99, 40)
loaded (6798, 99, 39)


In [5]:
train_labels = pd.read_csv("./data/train_list.csv")["label"].tolist()
len(train_labels), train_labels[:5]

(51088, ['right', 'right', 'right', 'right', 'right'])

In [6]:
valid_labels = pd.read_csv("./data/valid_list.csv")["label"].tolist()
len(valid_labels), valid_labels[:5]

(6798, ['right', 'right', 'right', 'right', 'right'])

In [7]:
target_labels = "yes, no, up, down, left, right, on, off, stop, go".split(", ")
target_labels

['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

In [8]:
train_labels = list(map(lambda x: x if x in target_labels else "unknown", train_labels))
valid_labels = list(map(lambda x: x if x in target_labels else "unknown", valid_labels))

In [9]:
label_counter = Counter(train_labels)
label_order, _ = list(zip(*label_counter.most_common(11)))
label_order

('unknown',
 'stop',
 'on',
 'go',
 'yes',
 'no',
 'right',
 'up',
 'down',
 'off',
 'left')

In [10]:
train_labels = list(map(lambda x: label_order.index(x), train_labels))
valid_labels = list(map(lambda x: label_order.index(x), valid_labels))

In [11]:
train_labels = to_categorical(train_labels, num_classes=11)
train_labels.shape

(51088, 11)

In [12]:
valid_labels = to_categorical(valid_labels, num_classes=11)
valid_labels.shape

(6798, 11)

## Using only MFCC as features

In [17]:
# Stop training well validation accuracy decreases
early_stop_callback = EarlyStopping(monitor='val_categorical_accuracy', 
                                    patience=2, mode='max')

### LSTM hidden_size=128

In [ ]:
train["mfcc"].shape

In [15]:
hidden_size = 128
batch_size = 32

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(hidden_size, input_shape=(99, 39)))
lstm_model.add(Dense(11))
lstm_model.add(Activation("softmax"))

lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])


In [ ]:
lstm_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size, 
               callbacks=[early_stop_callback],
               epochs=10, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/10
51088/51088 [==============================] - 278s 5ms/step - loss: 1.0576 - categorical_accuracy: 0.6836 - val_loss: 0.6602 - val_categorical_accuracy: 0.7954
Epoch 2/10
51088/51088 [==============================] - 275s 5ms/step - loss: 0.4842 - categorical_accuracy: 0.8484 - val_loss: 0.4075 - val_categorical_accuracy: 0.8741
Epoch 3/10
51088/51088 [==============================] - 275s 5ms/step - loss: 0.3232 - categorical_accuracy: 0.9000 - val_loss: 0.3557 - val_categorical_accuracy: 0.8910
Epoch 4/10
51088/51088 [==============================] - 275s 5ms/step - loss: 0.2644 - categorical_accuracy: 0.9192 - val_loss: 0.3114 - val_categorical_accuracy: 0.9036
Epoch 5/10
51088/51088 [==============================] - 275s 5ms/step - loss: 0.2324 - categorical_accuracy: 0.9286 - val_loss: 0.3105 - val_categorical_accuracy: 0.9042
Epoch 6/10
51088/51088 [==============================] - 276s 5ms/step - loss: 0.2061 - ca

validation accuracy = 0.9206

### LSTM hidden_size=256

In [14]:
hidden_size = 256
batch_size = 64

In [15]:
lstm_model = Sequential()
lstm_model.add(LSTM(hidden_size, input_shape=(99, 39)))
lstm_model.add(Dense(11))
lstm_model.add(Activation("softmax"))

lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])

In [16]:
lstm_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size,
               callbacks=[early_stop_callback],
               epochs=20, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/20
51088/51088 [==============================] - 155s 3ms/step - loss: 0.8811 - categorical_accuracy: 0.7419 - val_loss: 0.4507 - val_categorical_accuracy: 0.8588
Epoch 2/20
51088/51088 [==============================] - 152s 3ms/step - loss: 0.3233 - categorical_accuracy: 0.8999 - val_loss: 0.3195 - val_categorical_accuracy: 0.9019
Epoch 3/20
51088/51088 [==============================] - 155s 3ms/step - loss: 0.2296 - categorical_accuracy: 0.9291 - val_loss: 0.3078 - val_categorical_accuracy: 0.9081
Epoch 4/20
51088/51088 [==============================] - 157s 3ms/step - loss: 0.1836 - categorical_accuracy: 0.9429 - val_loss: 0.2755 - val_categorical_accuracy: 0.9210
Epoch 5/20
51088/51088 [==============================] - 157s 3ms/step - loss: 0.1534 - categorical_accuracy: 0.9518 - val_loss: 0.2631 - val_categorical_accuracy: 0.9239
Epoch 6/20
51088/51088 [==============================] - 156s 3ms/step - loss: 0.1365 - ca

final validation accuracy = 0.9297

### LSTM Hidden Size 512

In [20]:
hidden_size = 512
batch_size = 64

In [21]:
lstm_model = Sequential()
lstm_model.add(LSTM(hidden_size, input_shape=(99, 39)))
lstm_model.add(Dense(11))
lstm_model.add(Activation("softmax"))

lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])

In [22]:
lstm_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size,
               callbacks=[early_stop_callback],
               epochs=20, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/20
51088/51088 [==============================] - 142s 3ms/step - loss: 0.6837 - categorical_accuracy: 0.7954 - val_loss: 0.3307 - val_categorical_accuracy: 0.8985
Epoch 2/20
51088/51088 [==============================] - 141s 3ms/step - loss: 0.2497 - categorical_accuracy: 0.9231 - val_loss: 0.2755 - val_categorical_accuracy: 0.9164
Epoch 3/20
51088/51088 [==============================] - 141s 3ms/step - loss: 0.1799 - categorical_accuracy: 0.9456 - val_loss: 0.2463 - val_categorical_accuracy: 0.9247
Epoch 4/20
51088/51088 [==============================] - 144s 3ms/step - loss: 0.1444 - categorical_accuracy: 0.9556 - val_loss: 0.2208 - val_categorical_accuracy: 0.9325
Epoch 5/20
51088/51088 [==============================] - 142s 3ms/step - loss: 0.1226 - categorical_accuracy: 0.9621 - val_loss: 0.2258 - val_categorical_accuracy: 0.9319
Epoch 6/20
51088/51088 [==============================] - 141s 3ms/step - loss: 0.1109 - ca

validation accuracy 0.9356

### LSTM Hidden Size 1024

In [23]:
hidden_size = 1024
batch_size = 64

In [24]:
lstm_model = Sequential()
lstm_model.add(LSTM(hidden_size, input_shape=(99, 39)))
lstm_model.add(Dense(11))
lstm_model.add(Activation("softmax"))

lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])

In [25]:
lstm_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size,
               callbacks=[early_stop_callback],
               epochs=20, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/20
51088/51088 [==============================] - 141s 3ms/step - loss: 0.5498 - categorical_accuracy: 0.8344 - val_loss: 0.3052 - val_categorical_accuracy: 0.9092
Epoch 2/20
51088/51088 [==============================] - 151s 3ms/step - loss: 0.2084 - categorical_accuracy: 0.9349 - val_loss: 0.2383 - val_categorical_accuracy: 0.9256
Epoch 3/20
51088/51088 [==============================] - 149s 3ms/step - loss: 0.1500 - categorical_accuracy: 0.9543 - val_loss: 0.2133 - val_categorical_accuracy: 0.9390
Epoch 4/20
51088/51088 [==============================] - 140s 3ms/step - loss: 0.1240 - categorical_accuracy: 0.9609 - val_loss: 0.2153 - val_categorical_accuracy: 0.9348
Epoch 5/20
51088/51088 [==============================] - 140s 3ms/step - loss: 0.1008 - categorical_accuracy: 0.9688 - val_loss: 0.2102 - val_categorical_accuracy: 0.9412
Epoch 6/20
51088/51088 [==============================] - 140s 3ms/step - loss: 0.0955 - ca

validation accuracy 0.9415

### GRU hidden_size=128

In [ ]:
hidden_size = 128
batch_size = 64

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(hidden_size, input_shape=(99, 39)))
gru_model.add(Dense(11))
gru_model.add(Activation("softmax"))

gru_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])


In [ ]:
gru_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size, 
               callbacks=[early_stop_callback],
               epochs=10, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/10
51088/51088 [==============================] - 136s 3ms/step - loss: 1.1155 - categorical_accuracy: 0.6852 - val_loss: 0.6028 - val_categorical_accuracy: 0.8170
Epoch 2/10
51088/51088 [==============================] - 133s 3ms/step - loss: 0.4394 - categorical_accuracy: 0.8655 - val_loss: 0.3759 - val_categorical_accuracy: 0.8856
Epoch 3/10
51088/51088 [==============================] - 134s 3ms/step - loss: 0.2828 - categorical_accuracy: 0.9148 - val_loss: 0.3070 - val_categorical_accuracy: 0.9014
Epoch 4/10
51088/51088 [==============================] - 134s 3ms/step - loss: 0.2216 - categorical_accuracy: 0.9339 - val_loss: 0.2860 - val_categorical_accuracy: 0.9134
Epoch 5/10
51088/51088 [==============================] - 134s 3ms/step - loss: 0.1852 - categorical_accuracy: 0.9427 - val_loss: 0.2707 - val_categorical_accuracy: 0.9176
Epoch 6/10
51088/51088 [==============================] - 135s 3ms/step - loss: 0.1632 - ca

Valid Accuracy 0.9238

### GRU hidden_size=256

In [14]:
hidden_size = 256
batch_size = 64

In [15]:
gru_model = Sequential()
gru_model.add(GRU(hidden_size, input_shape=(99, 39)))
gru_model.add(Dense(11))
gru_model.add(Activation("softmax"))

gru_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])


In [18]:
gru_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size, 
               callbacks=[early_stop_callback],
               epochs=10, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/10
51088/51088 [==============================] - 122s 2ms/step - loss: 0.7809 - categorical_accuracy: 0.7745 - val_loss: 0.3384 - val_categorical_accuracy: 0.8984
Epoch 2/10
51088/51088 [==============================] - 119s 2ms/step - loss: 0.2541 - categorical_accuracy: 0.9211 - val_loss: 0.2808 - val_categorical_accuracy: 0.9141
Epoch 3/10
51088/51088 [==============================] - 121s 2ms/step - loss: 0.1686 - categorical_accuracy: 0.9479 - val_loss: 0.2604 - val_categorical_accuracy: 0.9228
Epoch 4/10
51088/51088 [==============================] - 119s 2ms/step - loss: 0.1324 - categorical_accuracy: 0.9588 - val_loss: 0.2405 - val_categorical_accuracy: 0.9316
Epoch 5/10
51088/51088 [==============================] - 117s 2ms/step - loss: 0.1109 - categorical_accuracy: 0.9651 - val_loss: 0.2486 - val_categorical_accuracy: 0.9245
Epoch 6/10
51088/51088 [==============================] - 117s 2ms/step - loss: 0.0960 - ca

validation accuracy 0.9316

### GRU hidden_size= 512

In [19]:
hidden_size = 512
batch_size = 64

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(hidden_size, input_shape=(99, 39)))
gru_model.add(Dense(11))
gru_model.add(Activation("softmax"))

gru_model.compile(optimizer="adam", loss="categorical_crossentropy",
                   metrics=['categorical_accuracy'])


In [ ]:
gru_model.fit(x=train["mfcc"], y=train_labels, 
               validation_data=(valid["mfcc"], valid_labels),
               batch_size=batch_size, 
               callbacks=[early_stop_callback],
               epochs=10, shuffle=True)

Train on 51088 samples, validate on 6798 samples
Epoch 1/10
51088/51088 [==============================] - 249s 5ms/step - loss: 0.6745 - categorical_accuracy: 0.8004 - val_loss: 0.3370 - val_categorical_accuracy: 0.8963
Epoch 2/10
51088/51088 [==============================] - 250s 5ms/step - loss: 0.2479 - categorical_accuracy: 0.9249 - val_loss: 0.2848 - val_categorical_accuracy: 0.9125
Epoch 3/10
51088/51088 [==============================] - 248s 5ms/step - loss: 0.1564 - categorical_accuracy: 0.9507 - val_loss: 0.2509 - val_categorical_accuracy: 0.9250
Epoch 5/10
51088/51088 [==============================] - 250s 5ms/step - loss: 0.1418 - categorical_accuracy: 0.9552 - val_loss: 0.2619 - val_categorical_accuracy: 0.9207
Epoch 6/10
51088/51088 [==============================] - 249s 5ms/step - loss: 0.1278 - categorical_accuracy: 0.9594 - val_loss: 0.2384 - val_categorical_accuracy: 0.9278
Epoch 7/10
51088/51088 [==============================] - 247s 5ms/step - loss: 0.1147 - ca

validation accuracy 0.9278